Modification of whisper-distil.ipynb to handle bulk transcriptions of an entire directory. Added a file renaming function to remove spaces from any audio filename (as ffmpeg will cut off after the space).

CAUTION: MAKE SURE TO BACKUP AUDIO FILES IF THEY HAVE SPACES IN THEIR NAMES AS THEY WILL BE RENAMED (AND THE METADATA ALTERED). For file names without spaces, the original file is not renamed and a copy is made in a compatible audio format.

In [3]:
import torch
torch.__version__

'2.2.1+cu121'

In [4]:
# need cuda for vastly faster transcription
#47 seconds for a full 30 min podcast! Insanely fast
torch.cuda.is_available()

True

In [5]:
# https://github.com/huggingface/distil-whisper
# conda activate py310
import torch
import subprocess
import os
import glob
import shutil
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(device)

# biggest model for highest quality
model_id = "distil-whisper/distil-large-v2"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, 
    torch_dtype=torch_dtype, 
    low_cpu_mem_usage=True, 
    use_safetensors=True,
    attn_implementation="flash_attention_2")        # updated
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

2024-03-17 16:30:55.117234: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-17 16:30:55.154468: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-17 16:30:55.864070: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-17 16:30:56.148394: I itex/core/wrapper/itex_cpu_wrapper.cc:52] Intel Extension for Tensorflow* AVX2 CPU backend is loaded.
2024-03-17 16:30:56.150678: W itex/core/wrapper/itex_gpu_wrapper.cc:32] Could not load dynamic library: libimf.so: cannot open sh

cuda:0


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# set podcasts=True to copy all downloaded podcasts to the whisper.cpp/samples directory
# example file location is from great Gnome Podcast app:  https://apps.gnome.org/en-GB/Podcasts/ 
podcasts = True

In [7]:
# rename all audio files with spaces in their name
# poe.com assisted code
# Specify the directory where the files are located
home_directory = os.path.expanduser("~")
directory = home_directory + '/machine_learning/whisper.cpp/samples/'
#directory = home_directory + '/Music/Voice_Memos/'

if podcasts:
    # copy podcast from subdirectories with spaces to whisper sample directory
    source_directory = home_directory + '/.var/app/org.gnome.Podcasts/data/gnome-podcasts/Downloads/'
    # Traverse the source directory and its subdirectories
    for root, directories, files in os.walk(source_directory):
        for file in files:
            source_path = os.path.join(root, file)
            destination_path = os.path.join(directory, file)
            # Copy the file to the destination directory
            shutil.copy2(source_path, destination_path)

# Get a list of all audio files, .m4a, .mp3, and .wav files, in the directory
# added *.m2a for podcasts
files = glob.glob(os.path.join(directory, '*.m4a')) + \
        glob.glob(os.path.join(directory, '*.mp3')) + \
        glob.glob(os.path.join(directory, '*.m2a')) + \
        glob.glob(os.path.join(directory, '*.ogg')) + \
        glob.glob(os.path.join(directory, '*.wav'))

# Iterate over the files (use this approach also for directory transcription)
# CAUTION: this overwrites files with spaces in their names such as 'Track 11.wav' to 'Track-11.wav' 
# by overwriting the file, it permits processing but alters the metadata to date file saved=today
for file in files:
    # If the file name contains a space
    if ' ' in file:
        # Replace the spaces with hyphens
        new_name = file.replace(' ', '-')
        # Rename the file
        os.rename(file, new_name)

In [8]:
# iterate over all audio files and transcribe them:
for file in files:
    audio_file = file
    # convert audio file to 16-bit wav format required by whisper
    output_file = audio_file + '-output.wav'
    print(audio_file)
    print(output_file)

    # convert audio_file then transcribe to text
    # overwrites existing file with same name with yes_command
    try:
        yes_command = f'echo "y" | '
        subprocess.run([yes_command + 'ffmpeg' + ' -i ' +  audio_file + ' -ar 16000 -ac 1 -c:a pcm_s16le ' 
                        + output_file], shell=True, check=True)
        print("Audio coverted successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Audio convertion failed with error {e.returncode}.")

    # pipeline
    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=15,
        batch_size=16,
        torch_dtype=torch_dtype,
        device=device
    )
    result_local = pipe(output_file)

    # save transcript as a .md file
    saved_txt=result_local["text"]
    f = open(output_file + ".md", "a")
    f.write(saved_txt)
    f.close()
    print("saved:", f)
        

/var/home/fraser/machine_learning/whisper.cpp/samples/5970.m2a
/var/home/fraser/machine_learning/whisper.cpp/samples/5970.m2a-output.wav
Audio coverted successfully.


ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place --cc=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libavresample   4.  0.  0 /  4.  0.  0

saved: <_io.TextIOWrapper name='/var/home/fraser/machine_learning/whisper.cpp/samples/5970.m2a-output.wav.md' mode='a' encoding='UTF-8'>
/var/home/fraser/machine_learning/whisper.cpp/samples/5971.m2a
/var/home/fraser/machine_learning/whisper.cpp/samples/5971.m2a-output.wav
Audio coverted successfully.


ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place --cc=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libavresample   4.  0.  0 /  4.  0.  0

saved: <_io.TextIOWrapper name='/var/home/fraser/machine_learning/whisper.cpp/samples/5971.m2a-output.wav.md' mode='a' encoding='UTF-8'>
/var/home/fraser/machine_learning/whisper.cpp/samples/5969.m2a
/var/home/fraser/machine_learning/whisper.cpp/samples/5969.m2a-output.wav
Audio coverted successfully.


...

saved: <_io.TextIOWrapper name='/var/home/fraser/machine_learning/whisper.cpp/samples/5971.m2a-output.wav-output.wav.md' mode='a' encoding='UTF-8'>
/var/home/fraser/machine_learning/whisper.cpp/samples/5969.m2a-output.wav
/var/home/fraser/machine_learning/whisper.cpp/samples/5969.m2a-output.wav-output.wav
Audio coverted successfully.


ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place --cc=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libavresample   4.  0.  0 /  4.  0.  0

saved: <_io.TextIOWrapper name='/var/home/fraser/machine_learning/whisper.cpp/samples/5969.m2a-output.wav-output.wav.md' mode='a' encoding='UTF-8'>
/var/home/fraser/machine_learning/whisper.cpp/samples/5967.m2a-output.wav
/var/home/fraser/machine_learning/whisper.cpp/samples/5967.m2a-output.wav-output.wav
Audio coverted successfully.


ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place --cc=/root/miniconda3/envs/conda_bld/conda-bld/ffmpeg_1635335682798/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libavresample   4.  0.  0 /  4.  0.  0

saved: <_io.TextIOWrapper name='/var/home/fraser/machine_learning/whisper.cpp/samples/5967.m2a-output.wav-output.wav.md' mode='a' encoding='UTF-8'>


In [9]:
# helper utility to combine all transcripts into one file (for ease of scanning podcasts)
# ai assisted
def combine_markdown_files(output_file_path, input_directory):
    markdown_files = glob.glob(input_directory + '*.md')

    with open(output_file_path, 'w') as output_file:
        for file_path in markdown_files:
            file_name = os.path.basename(file_path)
            with open(file_path, 'r') as input_file:
                output_file.write(f"## {file_name}\n\n")
                output_file.write(input_file.read())
                output_file.write('\n\n')  # Add newline between files

    print(f"Combined {len(markdown_files)} Markdown files into {output_file_path}")

# saves combined markdown files into directory of Jupyter notebook (not the input directory)
# saved as .txt file to ease send to kindle workflow
combine_markdown_files('podcast_transcripts.txt', directory)

Combined 8 Markdown files into podcast_transcripts.txt
